# Least Squares via L-BFGS in ensmallen

In [1]:
import numpy as np
import pyensmallen

In [2]:
# Generate some random data for linear regression
n, k = 1_000_000, 20
np.random.seed(42)
X = np.random.randn(n, k)
print(true_params := np.random.rand(k))
y = X @ true_params + np.random.randn(n)

[0.51639859 0.94598022 0.23380001 0.55162275 0.97811966 0.24254699
 0.64702478 0.70271041 0.26476461 0.77362184 0.7817448  0.36874977
 0.72697004 0.06518613 0.72705723 0.38967364 0.03826155 0.39386005
 0.0438693  0.72142769]


In [3]:
def linear_regression_objective(params, gradient):
    params = params.reshape(-1, 1)
    residuals = X @ params - y.reshape(-1, 1)
    objective = np.sum(residuals**2)
    grad = 2 * X.T @ residuals
    gradient[:] = grad.flatten()
    return objective

In [4]:
%%time
# Create an L-BFGS optimizer
optimizer = pyensmallen.L_BFGS()
# Optimize
result = optimizer.optimize(linear_regression_objective,
    np.zeros(k, dtype=np.float64))
print("pyensmallen estimated parameters:", result)
print("pyensmallen mean squared error:", np.mean((result - true_params)**2))

pyensmallen estimated parameters: [0.51556024 0.94691468 0.23404849 0.55121759 0.97818756 0.24338623
 0.64700696 0.70195589 0.26487498 0.77280983 0.78267599 0.36787315
 0.72791074 0.06571446 0.72615144 0.38766298 0.03820425 0.39468909
 0.04304362 0.72195013]
pyensmallen mean squared error: 6.530072513292412e-07
CPU times: user 4.96 s, sys: 163 ms, total: 5.12 s
Wall time: 342 ms


In [5]:
%%time
np_solution = np.linalg.lstsq(X, y, rcond=None)[0]

CPU times: user 2.82 s, sys: 52.3 ms, total: 2.87 s
Wall time: 433 ms


In [6]:

print("NumPy least squares solution:", np_solution)
print("Mean squared error (NumPy):", np.mean((np_solution - true_params)**2))


NumPy least squares solution: [0.51556024 0.94691468 0.23404849 0.55121759 0.97818756 0.24338623
 0.64700696 0.70195589 0.26487498 0.77280983 0.78267599 0.36787315
 0.72791074 0.06571446 0.72615144 0.38766298 0.03820425 0.39468909
 0.04304362 0.72195013]
Mean squared error (NumPy): 6.530072504955779e-07
